In [648]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import time
import itertools
from itertools import product
import torch

print('mps: ', torch.backends.mps.is_available())

options = {
 "WLSACCESSID":"a4353fb7-f95b-4075-b288-ca3f60983b36",
"WLSSECRET":"d894d460-2dac-4210-8c40-c91c68ecfb13",
"LICENSEID":2562382
}

# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Use the Metal backend
else:
    device = torch.device("cpu")  # Fallback to CPU

mps:  True


In [649]:
class BundledChoiceKP:
    def __init__(self,  num_agents , num_objects , lambda_star , random_seed = 4,  sigma = 1, max_capacity = 100):

        self.K_MOD = len(lambda_star)
        self.num_agents = num_agents
        self.num_objects = num_objects
        self.num_simulations = 100

        self.random_seed = random_seed

        self.agents_si = torch.kron(torch.ones(self.num_simulations, dtype = torch.int), torch.arange(self.num_agents))


        # Initialize stuff
   

        # True parameters
        self.lambda_star_np = lambda_star
        self.lambda_star = torch.tensor(lambda_star, device=device, dtype=torch.float)

        # Set manual seed
        torch.manual_seed(self.random_seed)

        ### Modular characteristics
        self.φ_i_j_k = torch.normal(0, 1, size=(self.num_agents, self.num_objects, self.K_MOD), device=device, dtype=torch.float)

        ### Knapsack Constraints
        self.weight_j = torch.randint(1, 100, (self.num_objects,), device=device, dtype = torch.int)
        self.capacity_i = torch.randint(1, max_capacity, (self.num_agents,), device=device, dtype=torch.int) 
        
    
        ### estimation
        self.φ_hat_k = None
        self.φ_hat_i_k = None
        self.value_LP = None

        self.eps_i_j = sigma * torch.normal(0, 1, size=(self.num_agents, self.num_objects), device=device, dtype=torch.float)
        self.eps_si_j = sigma * torch.normal(0, 1, size=(self.num_simulations * self.num_agents, self.num_objects), device=device, dtype=torch.float)

In [650]:
def linear_knapsack(self, idx, lambda_k , max_weight = None, p_j = None, eps_i_j = None):
    
    ### Compute the values
    values_i_j = self.φ_i_j_k[idx]  @  lambda_k 

    if p_j is not None:
        values_i_j -= p_j.unsqueeze(0)
    if eps_i_j is not None:
        values_i_j += eps_i_j

    if max_weight is None:
        max_weight = int(self.capacity_i[idx].max())
    
    ### Fill in the DP table
    V_i_j_w = torch.zeros((len(idx), self.num_objects +1, max_weight +1) ,device=device, dtype=torch.float)
    weight_states = torch.arange(max_weight + 1, device= device)

    for j in range(self.num_objects):
        V_i_j_w[:, j+1, :] = torch.where(self.weight_j[j] <= weight_states, 
                                    torch.maximum(values_i_j[:, j].unsqueeze(1) + V_i_j_w[:, j, weight_states - self.weight_j[j]],
                                                V_i_j_w[:, j, :]), 
                                        V_i_j_w[:, j, :])
        
    # print(V_i_j_w[torch.arange(len(idx)),-1, self.capacity_i[idx]])

    ### Backtrack to find the items
    residual_weight = self.capacity_i[idx]
    B_i_j_star = torch.zeros((len(idx), self.num_objects), device=device, dtype= bool)

    for j in range(self.num_objects,0,-1):

        pick_j = (V_i_j_w[torch.arange(len(idx)), j, residual_weight] > 
                    V_i_j_w[torch.arange(len(idx)), j-1, residual_weight])
        
        B_i_j_star[:, j-1] = pick_j
        residual_weight -= pick_j * self.weight_j[j-1]
        #residual_weight = torch.clamp(residual_weight, min=0)
    
    # print((values_i_j * B_i_j_star).sum(1))

    return B_i_j_star 

BundledChoiceKP.linear_knapsack = linear_knapsack

In [653]:
def generate_data(self, add_noise = False):

    noise = self.eps_i_j if add_noise else None

    B_i_j_star = self.linear_knapsack(torch.arange(self.num_agents), self.lambda_star, eps_i_j = noise)

    self.φ_hat_i_k = (self.φ_i_j_k * B_i_j_star.unsqueeze(2)).sum(1)
    self.φ_hat_k = self.φ_hat_i_k.sum(0)

    print("# characteristics: ",self.K_MOD)
    print("φ_hat_k: ", self.φ_hat_k.cpu().numpy())

BundledChoiceKP.generate_data = generate_data

In [654]:
example_pb = BundledChoiceKP( 10, 30, [1, 2, 3, 4, 2.5], max_capacity= 200)
# B_i_star = example_pb.linear_knapsack(torch.arange(500), example_pb.lambda_star)
# example_pb.generate_data()

In [655]:
def estimate_minmaxregret_BChoice(self, max_iters = 100 ,tol = 1e-2 ):
      with gp.Env(params=options) as env:
          with gp.Model(env=env) as model:
              ### Initialize
              # Create variables
              lambda_k = model.addVars(self.K_MOD - 1, lb= 0, ub = 1e8 , name="parameters")
              u_i = model.addVars(self.num_agents, lb = 0, ub = 1e8 , name="utilities")

              # Set objective and initial constraints
              model.setObjective(gp.quicksum( self.φ_hat_k[k+1] * lambda_k[k] for k in range(self.K_MOD - 1))
                                  - u_i.sum(), GRB.MAXIMIZE)

              model.addConstrs(
                            (u_i[i] >= gp.quicksum((self.φ_hat_i_k[i, k+1]) * lambda_k[k]  for k in range(self.K_MOD - 1))
                            + self.φ_hat_i_k[i, 0]
                            for i in range(self.num_agents))
                              )
              
              # Solve master problem 
              model.setParam('OutputFlag', 0)
              model.optimize()
              theta_solution = np.array(model.x)

              iter = 0
              while iter < max_iters:
                  print('################')
                  print(f"ITER: {iter}, parameters: {theta_solution[:self.K_MOD-1].round(3)}")

                  ### Pricing problem
                  lambda_k_iter = torch.cat((torch.tensor([1.0], device= device),
                                               torch.tensor(theta_solution[:self.K_MOD-1], device= device , dtype=torch.float32)))


                  B_star_i = self.linear_knapsack(torch.arange(self.num_agents), lambda_k_iter)

                  val_i =  (self.φ_i_j_k * B_star_i.unsqueeze(2)).sum(1) @ lambda_k_iter

                  ### Stop if certificate holds
                  certificate_i = val_i.cpu().numpy() - theta_solution[self.K_MOD-1:]
                  max_certificate = np.max(certificate_i)

                  print("Value of LP:   ", model.objVal)
                  print(f"Reduced cost: {max_certificate}")

                  if max_certificate < tol:
                    primal_solution = np.array(model.x)
                    dual_solution = np.array(model.pi)
                    print('#############################################')
                    print('#############################################')
                    print("Solution:       ", primal_solution[:self.K_MOD-1].round(3))
                    print("True parameters:", self.lambda_star_np[1:])
                    print('###############')
                    # print("Value of LP:   ", model.objVal)
                    # print("True value LP: ",self.value_LP)
                    print('#############################################')
                    # print('#############################################')
                    break

                  ### Master problem
                  # Add constraints

                  φ_i_k_star = (self.φ_i_j_k * B_star_i.unsqueeze(2)).sum(1)

                  model.addConstrs(
                            (u_i[i] >= gp.quicksum((φ_i_k_star[i, k+1]) * lambda_k[k]  for k in range(self.K_MOD - 1))
                            + φ_i_k_star[i, 0]
                            for i in range(self.num_agents))
                                  )
                  # Solve master problem
                  lambda_k.start = theta_solution[:self.K_MOD-1]
                  u_i.start = val_i
                  model.optimize()
                  theta_solution = np.array(model.x)

                  iter += 1


BundledChoiceKP.estimate_minmaxregret_BChoice = estimate_minmaxregret_BChoice

In [656]:
# example_pb.estimate_minmaxregret_BChoice(tol = 1e-4)

## Matching

In [657]:
def estimate_GMM_matching(self, max_iters = 100 ,tol = 1e-2 ):
      with gp.Env(params=options) as env:
          with gp.Model(env=env) as model:
              ### Initialize
              # Create variables
              lambda_k = model.addVars(self.K_MOD , lb= 0, ub = 1e6 , name="parameters")
              u_si = model.addVars(self.num_simulations * self.num_agents, lb = 0, ub = GRB.INFINITY , name="utilities")
              p_j = model.addVars(self.num_objects, lb = 0, ub = GRB.INFINITY , name="prices")

              # Set objective and initial constraints
              model.setObjective(gp.quicksum( self.φ_hat_k[k] * lambda_k[k] for k in range(self.K_MOD ))
                                  - (1 / self.num_simulations) * u_si.sum() - p_j.sum(), GRB.MAXIMIZE)

              model.addConstrs(
                            (u_si[si] + gp.quicksum(p_j[j] for j in np.where(self.B_i_j_hat[si // self.num_simulations])[0])
                             >= gp.quicksum((self.φ_hat_i_k[si // self.num_simulations, k]) * lambda_k[k]  for k in range(self.K_MOD))
                            + self.eps_si_j[si, self.B_i_j_hat[si // self.num_simulations]].sum()
                            for si in range(self.num_simulations * self.num_agents))
                            )
              
              # Solve master problem 
              model.setParam('OutputFlag', 0)
              model.optimize()
              theta_solution = np.array(model.x)

              iter = 0
              while iter < max_iters:
                  print('################')
                  print(f"ITER: {iter}, parameters: {theta_solution[:self.K_MOD].round(3)}")

                  ### Pricing problem
                  lambda_k_iter = torch.tensor(theta_solution[:self.K_MOD], device= device , dtype=torch.float32)
                  p_j_iter = torch.tensor(theta_solution[-self.num_objects:], device= device , dtype=torch.float32)

                  B_star_si = self.linear_knapsack(
                                                  self.agents_si, 
                                                  lambda_k_iter, 
                                                  eps_i_j = self.eps_si_j,
                                                  p_j = p_j_iter
                                                  )
                  
                  φ_si_k_star = (self.φ_i_j_k[example_pb.agents_si] * B_star_si.unsqueeze(2)).sum(1)

                  val_si =  φ_si_k_star @ lambda_k_iter

                  ### Stop if certificate holds
                  certificate_i = val_si.cpu().numpy() - theta_solution[self.K_MOD: -self.num_objects]
                  max_certificate = np.max(certificate_i)

                  print("Value of LP:   ", model.objVal)
                  print(f"Reduced cost: {max_certificate}")

                  if max_certificate < tol:
                    primal_solution = np.array(model.x)
                    dual_solution = np.array(model.pi)
                    print('#############################################')
                    print('#############################################')
                    print("Solution:       ", primal_solution[:self.K_MOD].round(3))
                    print("True parameters:", self.lambda_star_np)
                    print('###############')
                    # print("Value of LP:   ", model.objVal)
                    # print("True value LP: ",self.value_LP)
                    print('#############################################')
                    # print('#############################################')
                    break

                  ### Master problem
                  # Add constraints

                  # φ_i_k_star = (self.φ_i_j_k * B_star_i.unsqueeze(2)).sum(1)

                  model.addConstrs(
                            (u_si[si] + gp.quicksum(p_j[j] for j in np.where(B_star_si[si])[0])
                             >= gp.quicksum((φ_si_k_star[si, k]) * lambda_k[k]  for k in range(self.K_MOD ))
                            + self.eps_si_j[si, B_star_si[si]].sum()
                            for si in range(self.num_simulations * self.num_agents))
                              )
                  # Solve master problem
                  lambda_k.start = theta_solution[ : self.K_MOD]
                  u_si.start = val_si.cpu().numpy() 
                  model.optimize()
                  theta_solution = np.array(model.x)

                  iter += 1


BundledChoiceKP.estimate_GMM_matching = estimate_GMM_matching

In [658]:
def compute_assignment_large(self, max_iters = 100 ,tol = 1e-2 ):
      with gp.Env(params=options) as env:
          with gp.Model(env=env) as model:
              ### Initialize
              # Create variables
              u_si = model.addVars(self.num_simulations * self.num_agents, lb = 0, ub = GRB.INFINITY , name="utilities")
              p_j = model.addVars(self.num_objects, lb = 0, ub = GRB.INFINITY , name="prices")

              # Set objective and initial constraints
              model.setObjective( (1 / self.num_simulations) * u_si.sum() + p_j.sum(), GRB.MINIMIZE)
              
              # Solve master problem 
              model.setParam('OutputFlag', 0)
              model.optimize()
              theta_solution = np.array(model.x)

              iter = 0
              while iter < max_iters:
                  print('################')
                  ### Pricing problem
                  p_j_iter = torch.tensor(theta_solution[-self.num_objects:], device= device , dtype=torch.float32)

                  B_star_si = self.linear_knapsack(
                                                  self.agents_si, 
                                                  self.lambda_star, 
                                                  eps_i_j = self.eps_si_j,
                                                  p_j = p_j_iter
                                                  )
                 
                  φ_si_k_star = (self.φ_i_j_k[example_pb.agents_si] * B_star_si.unsqueeze(2)).sum(1)
               
                  val_si =  φ_si_k_star @ self.lambda_star + (self.eps_si_j * B_star_si).sum(1) - (B_star_si * p_j_iter).sum(1)

                  ### Stop if certificate holds
                  certificate_i = val_si.cpu().numpy() - theta_solution[ : -self.num_objects]
                  max_certificate = np.max(certificate_i)

                  print("Value of LP:   ", model.objVal)
                  print(f"Reduced cost: {max_certificate}")

                  if max_certificate < tol:
                    print('#############################################')
                    print("DONE")
                    return np.array(model.x), np.array(model.pi)
                    print('#############################################')
                    break

                  ### Master problem
                  # Add constraints
                  B_star_si = B_star_si.cpu().numpy()
                  
                  model.addConstrs(
                            (u_si[si] + gp.quicksum(p_j[j] for j in np.where(B_star_si[si])[0])
                             >= φ_si_k_star[si] @ self.lambda_star
                            + self.eps_si_j[si, B_star_si[si]].sum()
                            for si in range(self.num_simulations * self.num_agents))
                              )
                  
                  # Solve master problem
                  
                  u_si.start = val_si.cpu().numpy() 
                  model.optimize()
                  theta_solution = np.array(model.x)

                  iter += 1


BundledChoiceKP.compute_assignment_large = compute_assignment_large

In [659]:
primal_solution, dual_solution = example_pb.compute_assignment_large()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562382
Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
################
Value of LP:    0.0
Reduced cost: 52.910423278808594
begin add constraints
end add constraints
################
Value of LP:    94.53053581237793
Reduced cost: 32.1856803894043
begin add constraints
end add constraints
################
Value of LP:    153.233761947155
Reduced cost: 30.195255279541016
begin add constraints
end add constraints
################
Value of LP:    170.34780642032615
Reduced cost: 18.32301139831543
begin add constraints
end add constraints
################
Value of LP:    193.46719916343696
Reduced cost: 7.370992660522461
begin add constraints
end add constraints
################
Value of LP:    200.54177273750304
Reduced cost: 2.295408248901367
begin add constraints
end add constraints
################
Value of LP:    200.88439765453347
Reduced cost: 0.9164137840270996
